Install packages

In [7]:
!pip install tensorflow opencv-python mediapipe sklearn matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post4-py3-none-any.whl size=2973 sha256=e324f7e0e7ffb1d9f5fbdb9eff31a253eb7c10bcb7f1ab45f2baf8698b569d39
  Stored in directory: /root/.cache/pip/wheels/46/b8/69/4f7789b7d2fc0718a44cbc4ae5d40970a40c22c472ae50f0a1
Successfully built sklearn


Uncomment this part if you want to download the dataset from Kaggle and upload it on your Google drive

In [24]:
#!pip install -q kaggle

In [25]:
#from google.colab import files

In [26]:
#files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"albandietrich","key":"0fdd5775959905aea95b8bd6ef28d892"}'}

In [27]:
#!mkdir ~/.kaggle

In [28]:
#!cp kaggle.json ~/.kaggle/

In [29]:
#!chmod 600 ~/.kaggle/kaggle.json

In [30]:
#!kaggle datasets list

ref                                                                title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
arnabchaki/data-science-salaries-2023                              Data Science Salaries 2023 💸                       25KB  2023-04-13 09:55:16           9925        264  1.0              
asahu40/walmart-data-analysis-and-forcasting                       Walmart Data Analysis and Forcasting              122KB  2023-04-26 07:07:03           1302         32  1.0              
ahmedshahriarsakib/usa-real-estate-dataset                         USA Real Estate Dataset                             1MB  2023-04-30 03:51:36           1067         29  1.0              
iammustafatz/diabetes-prediction-dataset               

In [41]:
#!kaggle datasets download -d grassknoted/asl-alphabet

100% 1.02G/1.03G [00:36<00:00, 30.4MB/s]
100% 1.03G/1.03G [00:36<00:00, 30.3MB/s]


In [43]:
#!unzip asl-alphabet.zip

Streaming output truncated to the last 5000 lines.
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing19.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing190.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1900.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1901.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1902.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1903.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1904.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1905.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1906.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1907.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1908.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1909.jpg  
  inflating: asl_alphabet_tr

Import packages

In [1]:
import mediapipe as mp

In [2]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
import numpy as np
import pandas as pd


In [3]:
import tensorflow
from tensorflow.keras.utils import to_categorical

In [4]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler    

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
import PIL

# Convert images to live image landmarks version:
Below takes in an array where each element is the file name of an image in the folder data/unlabled/. The script reads the image, runs a hand detection algorithm, then overlays the results and saves the output to the folder data/labeled/ with the original file name + "_marked.png"


TODO: Save the keypoints into a separate csv to create a training dataset

Now, we want the relative normalized coordinates of thes hand so it is invariant to location and image size!

In [7]:
alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

In [8]:
len(alphabet)

26

## Feature Extractor Work

In [9]:
def get_features(path, labels, N_imgs):
    
    x_data = []
    
    y_data = []
    
    for lab in labels:
        with mp_hands.Hands(
            static_image_mode=True,
            max_num_hands=2,
            min_detection_confidence=0.5) as hands:
          for i in range(1,N_imgs):
            file = path + lab + '/'+ lab + str(i) + ".jpg"
            print(file)
            #print("iteration",idx,"file,",file)
            # Read an image, flip it around y-axis for correct handedness output (see
            # above).

            # image = cv2.flip(cv2.imread(file), 1)
            image = cv2.imread(file)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Convert the BGR image to RGB before processing.
            results = hands.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            # Extract the hand landmarks
            landmarks = []
            if results.multi_hand_landmarks:
                print('Sample ', i, 'of label ', lab)
                for hand_landmarks in results.multi_hand_landmarks:
                    # Get the bounding box of the hand
                    x_min, y_min, x_max, y_max = float('inf'), float('inf'), float('-inf'), float('-inf')
                    for landmark in hand_landmarks.landmark:
                        x, y = landmark.x, landmark.y
                        x_min = min(x_min, x)
                        y_min = min(y_min, y)
                        x_max = max(x_max, x)
                        y_max = max(y_max, y)

                    # Normalize the coordinates with respect to the bounding box of the hand
                    for landmark in hand_landmarks.landmark:
                        x, y, z = landmark.x, landmark.y, landmark.z
                        x_norm = (x - x_min) / (x_max - x_min)
                        y_norm = (y - y_min) / (y_max - y_min)
                        landmarks.append([x_norm, y_norm, z])
                                
                # Convert the landmarks to a feature vector
                feature_vector = np.array(landmarks)

                if np.array(landmarks).flatten().shape[0] == 63: # Remove feature_vector of shape 126 due extra hand detection
                    # Print the feature vector
                    x_data.append(feature_vector)
                    y_data.append(alphabet.index(lab))
            
    return x_data, y_data

Connect to Google drive

In [10]:
import os
from google.colab import drive

drive.mount('/content/drive/')
os.chdir('drive/My Drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Import the dataset. Choose the letter you want to use for the model and choose the number of samples per letter (samples_per_symbol)

In [13]:
path = "asl_alphabet_train/asl_alphabet_train/"

labels_ = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']

samples_per_symbol = 1000

x_data, y_data = get_features(path, labels_, samples_per_symbol)

Streaming output truncated to the last 5000 lines.
asl_alphabet_train/asl_alphabet_train/G/G400.jpg
Sample  400 of label  G
asl_alphabet_train/asl_alphabet_train/G/G401.jpg
Sample  401 of label  G
asl_alphabet_train/asl_alphabet_train/G/G402.jpg
Sample  402 of label  G
asl_alphabet_train/asl_alphabet_train/G/G403.jpg
Sample  403 of label  G
asl_alphabet_train/asl_alphabet_train/G/G404.jpg
Sample  404 of label  G
asl_alphabet_train/asl_alphabet_train/G/G405.jpg
Sample  405 of label  G
asl_alphabet_train/asl_alphabet_train/G/G406.jpg
Sample  406 of label  G
asl_alphabet_train/asl_alphabet_train/G/G407.jpg
Sample  407 of label  G
asl_alphabet_train/asl_alphabet_train/G/G408.jpg
Sample  408 of label  G
asl_alphabet_train/asl_alphabet_train/G/G409.jpg
Sample  409 of label  G
asl_alphabet_train/asl_alphabet_train/G/G410.jpg
Sample  410 of label  G
asl_alphabet_train/asl_alphabet_train/G/G411.jpg
Sample  411 of label  G
asl_alphabet_train/asl_alphabet_train/G/G412.jpg
Sample  412 of label  G


Check if no empty array is present in our dataset

In [14]:
ii = 0
for el in x_data:
    if el == []:
        ii += 1
print(ii)

0


<ipython-input-14-1b65b0da7177>:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if el == []:


In [15]:
x_data = np.array(x_data)
y_data = np.array(y_data)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

The number of classes corresponds to the number of letter we chose in our dataset

In [17]:
NUM_CLASSES = len(labels_)

Create the model

In [18]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(21, 3)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))

Load model (uncomment)

In [20]:
# Loads the weights
#checkpoint_path = "cp.ckpt"
#model.load_weights(checkpoint_path)

In [21]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Create the training and test dataset

In [22]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.05)

Convert into one-hot encoding

In [23]:
y_train = to_categorical(y_train).astype(int)

Save model

In [38]:
import os
import tensorflow as tf

checkpoint_path = "ADLProject/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)



Train the model

In [26]:
model.fit(X_train, y_train, epochs=10, callbacks=[cp_callback])

Epoch 1/10
216/217 [============================>.] - ETA: 0s - loss: 0.0715 - categorical_accuracy: 0.9754
Epoch 1: saving model to cp.ckpt
217/217 [==============================] - 18s 81ms/step - loss: 0.0714 - categorical_accuracy: 0.9755
Epoch 2/10
 13/217 [>.............................] - ETA: 13s - loss: 0.2171 - categorical_accuracy: 0.9303

KeyboardInterrupt: ignored

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 21, 64)            17408     
                                                                 
 lstm_1 (LSTM)               (None, 21, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 9)                 297       
                                                                 
Total params: 172,169
Trainable params: 172,169
Non-trai

Save model

In [39]:
os.listdir(checkpoint_dir)

['archive']

Make some prediction

In [30]:
res = model.predict(X_test)

Exception ignored in: <function _xla_gc_callback at 0x7f8f03cf8a60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


12/12 [==============================] - 43s 20ms/step


Go from one hot encoding to number of class

In [188]:
res_new = []

for el in res:
  res_new.append(np.argmax(el))

res_new = np.array(res_new)

Now let us try to run that in real time with our webcam. To do so, as we are running this code on Google Colab, several steps are necessary to access the webcam (see below).

In [31]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [32]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [33]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [35]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
  while True:
      letter = ''
      js_reply = video_frame(label_html, bbox)
      if not js_reply:
          break

      # convert JS response to OpenCV Image
      image = js_to_image(js_reply["img"])

      image.flags.writeable = False
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      results = hands.process(image)

      # Draw the hand annotations on the image.
      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
      
      landmarks = []
      letter = ''
      if results.multi_hand_landmarks:
          for hand_landmarks in results.multi_hand_landmarks:
              mp_drawing.draw_landmarks(
                  image,
                  hand_landmarks,
                  mp_hands.HAND_CONNECTIONS,
                  mp_drawing_styles.get_default_hand_landmarks_style(),
                  mp_drawing_styles.get_default_hand_connections_style())
              
              # Get the bounding box of the hand
              x_min, y_min, x_max, y_max = float('inf'), float('inf'), float('-inf'), float('-inf')
              for landmark in hand_landmarks.landmark:
                  x, y = landmark.x, landmark.y
                  x_min = min(x_min, x)
                  y_min = min(y_min, y)
                  x_max = max(x_max, x)
                  y_max = max(y_max, y)

              # Normalize the coordinates with respect to the bounding box of the hand
              for landmark in hand_landmarks.landmark:
                  x, y, z = landmark.x, landmark.y, landmark.z
                  x_norm = (x - x_min) / (x_max - x_min)
                  y_norm = (y - y_min) / (y_max - y_min)
                  landmarks.append([x_norm, y_norm, z])

          # Convert the landmarks to a feature vector
          x_t = np.array(landmarks)
        
          
          if x_t.flatten().shape[0] == 63:

              res_ = model.predict(np.array([x_t]), verbose = 0)

              index_class = np.argmax(res_)

              letter = alphabet[index_class]

      label_html = letter


<IPython.core.display.Javascript object>